In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Multiply
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPool3D

In [4]:
class ColoredMNISTEnvironments():
    
    def __init__(self):
        
        self.__load_initial_data()
        self.__create_envs()
        self.__create_validation_envs()

    def __load_initial_data(self):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        # convert to RGB
        x_train = np.stack((x_train,)*3, axis=-1)
        x_test = np.stack((x_test,)*3, axis=-1)

        # normalize
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        # binary label
        y_train = (y_train < 5).astype(int)
        y_test = (y_test < 5).astype(int)
        
        self.original_data = {
            'x_train':x_train,
            'x_test':x_test,
            'y_train':y_train,
            'y_test':y_test
        }
        
    def __create_envs(self):
        k=10**4
        self.e1 = self.__create_env(self.original_data['x_train'][:k], 
                                    self.original_data['y_train'][:k], .1)
        self.e2 = self.__create_env(self.original_data['x_train'][k:2*k], 
                                    self.original_data['y_train'][k:2*k], .2)
        self.e3 = self.__create_env(self.original_data['x_train'][2*k:3*k], 
                                    self.original_data['y_train'][2*k:3*k], .9)
        
    def __create_validation_envs(self):
        k=10**4
        i=3*k
        self.e11 = self.__create_env(self.original_data['x_train'][i:i+k], 
                                     self.original_data['y_train'][i:i+k], .1)
        self.e22 = self.__create_env(self.original_data['x_train'][i+k:i+2*k], 
                                     self.original_data['y_train'][i+k:i+2*k], .2)
        self.e33 = self.__create_env(self.original_data['x_train'][i+2*k:i+3*k], 
                                     self.original_data['y_train'][i+2*k:i+3*k], .9)
        
    def __create_env(self, x, y, e, labelflip_proba=.25):
        x = x.copy()
        y = y.copy()

        y = np.logical_xor(
            y,
            (np.random.random(size=len(y)) < labelflip_proba).astype(int)
        ).astype(int)

        color = np.logical_xor(
            y,
            (np.random.random(size=len(y)) < e).astype(int)
        )

        x[color, :, :, 2] = 0
        x[color, :, :, 1] = 0
        return tf.data.Dataset.from_tensor_slices((x, y))

In [90]:
def get_model(n_final_units=32, compile=False):
    
    input_images = Input(shape=(28, 28, 3))
    
    cnn = Conv2D(32, kernel_size=(3, 3),
                 activation='relu')(input_images)
    cnn = Conv2D(64, (3, 3), activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=(2, 2))(cnn)
    cnn = Dropout(0.25)(cnn)
    cnn = Flatten()(cnn)
    
    env1 = Dense(32, activation='relu')(cnn)
    env1 = Dropout(0.5)(env1)
    env1 = Dense(1, name='env1')(env1)
        
    model = Model(
        inputs=[input_images],
        outputs=[env1]
    )
    
    if compile:
        model.compile(
            loss=[
                tf.keras.losses.binary_crossentropy,
            ],
            optimizer=tf.keras.optimizers.Adadelta(),
            metrics=['accuracy']
        )
    return model

In [96]:
from collections import defaultdict
from datetime import datetime

class IRMModel(object):
    
    def __init__(self, model = get_model(), optimizer = tf.keras.optimizers.Adam()):
        self.model = model
        self.optimizer = optimizer
        self.envs = ColoredMNISTEnvironments()
        self.dummy = tf.convert_to_tensor([1.])
        self.loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        self.logs = defaultdict(list)
        self.logdir = "/home/e.diemert/tflogs/scalars/" + datetime.now().strftime("%Y-%m-%d-%H_%M_%S")
        print("logging to", self.logdir)
        self.file_writer = tf.summary.create_file_writer(self.logdir + "/metrics")
        self.file_writer.set_as_default()

        
    def evaluate(self, env):
        accuracy = tf.keras.metrics.Accuracy()
        loss = tf.keras.metrics.BinaryCrossentropy()
        per_batch_penalties = []
        for (batch, (x, y)) in enumerate(env):
            with tf.GradientTape() as tape:
                tape.watch(self.dummy)
                logits = self.model(x, training=False)
                dummy_loss = self.loss(y, logits) * self.dummy
            batch_grads = tape.gradient(dummy_loss, [self.dummy])
            per_batch_penalties += [
                tf.math.square(
                    dummy_loss * batch_grads
                )
            ]
            loss.update_state(y, logits)
            accuracy.update_state(y, tf.math.greater(tf.keras.activations.sigmoid(logits), .5))
        return loss.result().numpy(), accuracy.result().numpy(), tf.reduce_mean(per_batch_penalties)
    
    def compute_penalty(self, x, y):
        with tf.GradientTape() as tape:
            tape.watch(self.dummy)
            dummy_logits = self.model(x, training=True)
            dummy_loss = self.loss(y, dummy_logits * self.dummy)
        dummy_grads = tape.gradient(dummy_loss, self.dummy)
        dummy_penalty = dummy_grads ** 2
        return dummy_loss, dummy_penalty
        
    def batch_gradients(self, x, y, penalty):
        with tf.GradientTape() as tape:
            tape.watch(penalty)
            logits = self.model(x, training=True)
            loss_value = self.loss(y, logits) + penalty
        grads = tape.gradient(loss_value, self.model.trainable_variables)
        return loss_value, grads
    
    def do_evaluations(self, epoch, print_=True, batch_size=128):
        if print_:
            print('-'*80)
            print("epoch:", epoch)
        ood_loss, ood_acc, ood_penalty = self.evaluate(self.envs.e3.shuffle(2*batch_size).batch(batch_size))
        self.logs['ood-loss'] += [ood_loss]
        self.logs['ood-acc'] += [ood_acc]
        self.logs['ood-penalty'] += [ood_penalty.numpy()]
        self.log_event('ood_loss', ood_loss, epoch)
        self.log_event('ood_acc', ood_acc, epoch)
        self.log_event('ood_pen', ood_penalty.numpy(), epoch)
        if print_:
            print('ood  loss %.5f acc: %.3f'%(ood_loss, ood_acc))   
        for env_name, env in (('e11',self.envs.e11.shuffle(2*batch_size).batch(batch_size)), 
                              ('e22',self.envs.e22.shuffle(2*batch_size).batch(batch_size)),):
            env_loss, env_acc, env_penalty = self.evaluate(env)
            self.logs[env_name+'-test-loss'] += [env_loss]
            self.logs[env_name+'-test-acc'] += [env_acc]
            self.log_event(env_name+'_loss', env_loss, epoch)
            self.log_event(env_name+'_acc', env_acc, epoch)
            self.log_event(env_name+'_pen', env_penalty.numpy(), epoch)
            if print_:
                print('%s loss %.5f acc: %.3f'%(env_name, env_loss, env_acc))
        if print_:
            print('-'*80)

    def log_event(self, event, value, epoch):
        tf.summary.scalar(event, data=value, step=epoch)
                
    def train(self, epochs, lambda_, batch_size=128):
        for epoch in range(epochs):
            d1 = self.envs.e1.shuffle(2*batch_size).batch(batch_size).__iter__()
            d2 = self.envs.e2.shuffle(2*batch_size).batch(batch_size).__iter__()
            batch = 0
            while True:
                try:
                    x1, y1 = d1.next()
                    x2, y2 = d2.next()
                    l1d, pen1 = self.compute_penalty(x1, y1)
                    l2d, pen2 = self.compute_penalty(x2, y2)
                    self.logs['e1'+'-train-penalty'] += pen1.numpy()
                    self.logs['e2'+'-train-penalty'] += pen2.numpy()
                    pen = tf.reduce_mean([pen1, pen2])
                    l1, grads1 = self.batch_gradients(x1, y1, lambda_(epoch) * pen)
                    l2, grads2 = self.batch_gradients(x2, y2, lambda_(epoch) * pen)
                    self.logs['e1'+'-train-loss'] += [l1]
                    self.logs['e2'+'-train-loss'] += [l2]
                    grads = [ tf.reduce_mean([grads1[_], grads2[_]], axis=0) for _ in range(len(grads1)) ]
                    self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
                    if not batch % 10:
                        print("%4d"%batch, 
                              "tr-l1: %.5f"%l1d.numpy(), "tr-p1: %.5f"%(pen1.numpy()*lambda_(epoch)), 
                              "tr-l2: %.5f"%l2d.numpy(), "tr-p2: %.5f"%(pen2.numpy()*lambda_(epoch)))
                    batch += 1
                except StopIteration:
                    break
            self.do_evaluations(epoch, batch_size=batch_size)

In [ ]:
irm = IRMModel(model = get_model(n_final_units=128), 
               optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3))
def lambda_scheduler(epoch):
    if epoch < 2:
        return 0
    elif epoch < 5:
        return 10000
    else:
        return 100000
irm.train(500, lambda_scheduler, batch_size=256)

logging to /home/e.diemert/tflogs/scalars/2020-05-04-10_04_58
   0 tr-l1: 0.70072 tr-p1: 0.00000 tr-l2: 0.70116 tr-p2: 0.00000
  10 tr-l1: 0.36983 tr-p1: 0.00000 tr-l2: 0.49441 tr-p2: 0.00000
  20 tr-l1: 0.35040 tr-p1: 0.00000 tr-l2: 0.46919 tr-p2: 0.00000
  30 tr-l1: 0.33255 tr-p1: 0.00000 tr-l2: 0.55774 tr-p2: 0.00000
--------------------------------------------------------------------------------
epoch: 0
ood  loss 13.37549 acc: 0.105
e11 loss 1.50737 acc: 0.899
e22 loss 2.98953 acc: 0.799
--------------------------------------------------------------------------------
   0 tr-l1: 0.33547 tr-p1: 0.00000 tr-l2: 0.51481 tr-p2: 0.00000
  10 tr-l1: 0.31002 tr-p1: 0.00000 tr-l2: 0.46249 tr-p2: 0.00000
  20 tr-l1: 0.27567 tr-p1: 0.00000 tr-l2: 0.47504 tr-p2: 0.00000
  30 tr-l1: 0.31999 tr-p1: 0.00000 tr-l2: 0.51239 tr-p2: 0.00000
--------------------------------------------------------------------------------
epoch: 1
ood  loss 12.92745 acc: 0.104
e11 loss 1.45737 acc: 0.899
e22 loss 2.84

  30 tr-l1: 0.30153 tr-p1: 545.50385 tr-l2: 0.45698 tr-p2: 692.26664
--------------------------------------------------------------------------------
epoch: 15
ood  loss 11.79775 acc: 0.117
e11 loss 1.37090 acc: 0.896
e22 loss 2.64297 acc: 0.798
--------------------------------------------------------------------------------
   0 tr-l1: 0.28847 tr-p1: 0.60381 tr-l2: 0.53831 tr-p2: 4952.03920
  10 tr-l1: 0.29406 tr-p1: 705.40584 tr-l2: 0.50705 tr-p2: 901.00328
  20 tr-l1: 0.25357 tr-p1: 356.05356 tr-l2: 0.38511 tr-p2: 128.96664
  30 tr-l1: 0.28238 tr-p1: 725.88669 tr-l2: 0.37368 tr-p2: 109.93241
--------------------------------------------------------------------------------
epoch: 16
ood  loss 11.62670 acc: 0.112
e11 loss 1.36381 acc: 0.897
e22 loss 2.66786 acc: 0.799
--------------------------------------------------------------------------------
   0 tr-l1: 0.28678 tr-p1: 57.98080 tr-l2: 0.51039 tr-p2: 2614.72501
  10 tr-l1: 0.29903 tr-p1: 1106.24805 tr-l2: 0.40410 tr-p2: 16.83815
  

e22 loss 2.67789 acc: 0.797
--------------------------------------------------------------------------------
   0 tr-l1: 0.18335 tr-p1: 205.57288 tr-l2: 0.41740 tr-p2: 1786.93626
  10 tr-l1: 0.21281 tr-p1: 1336.05860 tr-l2: 0.47017 tr-p2: 859.36571
  20 tr-l1: 0.28367 tr-p1: 0.20964 tr-l2: 0.32599 tr-p2: 8.93943
  30 tr-l1: 0.28622 tr-p1: 108.31493 tr-l2: 0.34536 tr-p2: 26.21097
--------------------------------------------------------------------------------
epoch: 31
ood  loss 10.70934 acc: 0.164
e11 loss 1.43514 acc: 0.885
e22 loss 2.54077 acc: 0.795
--------------------------------------------------------------------------------
   0 tr-l1: 0.24231 tr-p1: 62.63378 tr-l2: 0.41226 tr-p2: 702.67166
  10 tr-l1: 0.24757 tr-p1: 516.46819 tr-l2: 0.30247 tr-p2: 58.34220
  20 tr-l1: 0.29123 tr-p1: 85.99638 tr-l2: 0.34043 tr-p2: 2.22107
  30 tr-l1: 0.25038 tr-p1: 247.81700 tr-l2: 0.29174 tr-p2: 24.62973
--------------------------------------------------------------------------------
epoch: 32

  20 tr-l1: 0.22106 tr-p1: 466.97734 tr-l2: 0.32952 tr-p2: 180.57694
  30 tr-l1: 0.20948 tr-p1: 153.31629 tr-l2: 0.26777 tr-p2: 0.15357
--------------------------------------------------------------------------------
epoch: 46
ood  loss 10.96441 acc: 0.197
e11 loss 1.50680 acc: 0.875
e22 loss 2.67473 acc: 0.790
--------------------------------------------------------------------------------
   0 tr-l1: 0.15280 tr-p1: 342.40573 tr-l2: 0.32546 tr-p2: 454.20723
  10 tr-l1: 0.22328 tr-p1: 106.79790 tr-l2: 0.27782 tr-p2: 63.88446
  20 tr-l1: 0.18831 tr-p1: 285.87505 tr-l2: 0.32652 tr-p2: 370.10768
  30 tr-l1: 0.22929 tr-p1: 116.40809 tr-l2: 0.26782 tr-p2: 38.47552
--------------------------------------------------------------------------------
epoch: 47
ood  loss 10.62108 acc: 0.197
e11 loss 1.58068 acc: 0.875
e22 loss 2.74603 acc: 0.790
--------------------------------------------------------------------------------
   0 tr-l1: 0.20080 tr-p1: 79.99035 tr-l2: 0.28331 tr-p2: 39.67090
  10 tr

e22 loss 2.80991 acc: 0.791
--------------------------------------------------------------------------------
   0 tr-l1: 0.15480 tr-p1: 74.66906 tr-l2: 0.23349 tr-p2: 0.04006
  10 tr-l1: 0.12700 tr-p1: 301.91904 tr-l2: 0.19955 tr-p2: 131.80418
  20 tr-l1: 0.15746 tr-p1: 198.48347 tr-l2: 0.23818 tr-p2: 1.01023
  30 tr-l1: 0.15981 tr-p1: 137.40207 tr-l2: 0.21743 tr-p2: 74.88588
--------------------------------------------------------------------------------
epoch: 62
ood  loss 10.49025 acc: 0.224
e11 loss 1.73346 acc: 0.868
e22 loss 2.73857 acc: 0.788
--------------------------------------------------------------------------------
   0 tr-l1: 0.14093 tr-p1: 250.59376 tr-l2: 0.22136 tr-p2: 7.49548
  10 tr-l1: 0.14881 tr-p1: 66.10000 tr-l2: 0.25422 tr-p2: 30.51900
  20 tr-l1: 0.12048 tr-p1: 664.13381 tr-l2: 0.23131 tr-p2: 86.68506
  30 tr-l1: 0.12795 tr-p1: 251.61100 tr-l2: 0.21394 tr-p2: 3.60102
--------------------------------------------------------------------------------
epoch: 63
ood

...

--------------------------------------------------------------------------------
epoch: 422
ood  loss 11.64721 acc: 0.223
e11 loss 1.87055 acc: 0.869
e22 loss 3.05372 acc: 0.787
--------------------------------------------------------------------------------
   0 tr-l1: 0.04279 tr-p1: 0.13098 tr-l2: 0.08169 tr-p2: 0.31910
  10 tr-l1: 0.02106 tr-p1: 15.13622 tr-l2: 0.07273 tr-p2: 14.01930
  20 tr-l1: 0.06619 tr-p1: 18.86701 tr-l2: 0.06405 tr-p2: 5.08392
  30 tr-l1: 0.04166 tr-p1: 17.31278 tr-l2: 0.08100 tr-p2: 6.78814
--------------------------------------------------------------------------------
epoch: 423
ood  loss 11.50679 acc: 0.227
e11 loss 1.88492 acc: 0.865
e22 loss 3.18259 acc: 0.786
--------------------------------------------------------------------------------
   0 tr-l1: 0.03335 tr-p1: 16.01842 tr-l2: 0.07428 tr-p2: 0.80837
  10 tr-l1: 0.05070 tr-p1: 23.62766 tr-l2: 0.06651 tr-p2: 2.40721
  20 tr-l1: 0.03156 tr-p1: 13.40241 tr-l2: 0.04926 tr-p2: 8.26005
  30 tr-l1: 0.03954 

  20 tr-l1: 0.02301 tr-p1: 26.96903 tr-l2: 0.05784 tr-p2: 0.03134
  30 tr-l1: 0.03654 tr-p1: 8.17213 tr-l2: 0.09434 tr-p2: 41.71054
--------------------------------------------------------------------------------
epoch: 438
ood  loss 11.75044 acc: 0.213
e11 loss 1.86408 acc: 0.871
e22 loss 3.11501 acc: 0.788
--------------------------------------------------------------------------------
   0 tr-l1: 0.03397 tr-p1: 19.88304 tr-l2: 0.06716 tr-p2: 8.16642
  10 tr-l1: 0.02459 tr-p1: 21.20830 tr-l2: 0.03993 tr-p2: 1.91947
  20 tr-l1: 0.05119 tr-p1: 12.45564 tr-l2: 0.06343 tr-p2: 20.59842
  30 tr-l1: 0.06134 tr-p1: 6.62398 tr-l2: 0.11530 tr-p2: 26.28859
--------------------------------------------------------------------------------
epoch: 439
ood  loss 11.64973 acc: 0.219
e11 loss 1.88625 acc: 0.869
e22 loss 3.17222 acc: 0.785
--------------------------------------------------------------------------------
   0 tr-l1: 0.02979 tr-p1: 13.48315 tr-l2: 0.05438 tr-p2: 2.47015
  10 tr-l1: 0.02569

   0 tr-l1: 0.03668 tr-p1: 21.61534 tr-l2: 0.07061 tr-p2: 0.16214
  10 tr-l1: 0.03818 tr-p1: 0.00450 tr-l2: 0.05239 tr-p2: 2.10628
  20 tr-l1: 0.03481 tr-p1: 40.87291 tr-l2: 0.05886 tr-p2: 1.47783
  30 tr-l1: 0.06138 tr-p1: 13.46945 tr-l2: 0.07813 tr-p2: 5.48107
--------------------------------------------------------------------------------
epoch: 454
ood  loss 11.60441 acc: 0.220
e11 loss 1.84338 acc: 0.869
e22 loss 3.09554 acc: 0.787
--------------------------------------------------------------------------------
   0 tr-l1: 0.05611 tr-p1: 2.34385 tr-l2: 0.08338 tr-p2: 26.12002
  10 tr-l1: 0.05427 tr-p1: 5.11166 tr-l2: 0.09405 tr-p2: 23.15212
  20 tr-l1: 0.04581 tr-p1: 2.23319 tr-l2: 0.04735 tr-p2: 27.80950
  30 tr-l1: 0.03572 tr-p1: 2.40684 tr-l2: 0.06655 tr-p2: 2.00795
--------------------------------------------------------------------------------
epoch: 455
ood  loss 11.52041 acc: 0.228
e11 loss 1.87907 acc: 0.866
e22 loss 3.11277 acc: 0.785
-------------------------------------

ood  loss 11.52864 acc: 0.223
e11 loss 1.99724 acc: 0.864
e22 loss 3.07789 acc: 0.785
--------------------------------------------------------------------------------
   0 tr-l1: 0.03705 tr-p1: 14.59896 tr-l2: 0.06703 tr-p2: 8.24317
  10 tr-l1: 0.05978 tr-p1: 0.73254 tr-l2: 0.06634 tr-p2: 2.17281
  20 tr-l1: 0.02659 tr-p1: 1.20363 tr-l2: 0.08677 tr-p2: 83.36927
  30 tr-l1: 0.04479 tr-p1: 3.25851 tr-l2: 0.14854 tr-p2: 704.53007
--------------------------------------------------------------------------------
epoch: 470
ood  loss 11.37013 acc: 0.236
e11 loss 1.94472 acc: 0.867
e22 loss 3.11775 acc: 0.784
--------------------------------------------------------------------------------
   0 tr-l1: 0.03213 tr-p1: 19.64341 tr-l2: 0.06221 tr-p2: 1.43397
  10 tr-l1: 0.03916 tr-p1: 0.23307 tr-l2: 0.10498 tr-p2: 204.56833
  20 tr-l1: 0.03920 tr-p1: 14.00214 tr-l2: 0.06170 tr-p2: 1.53051
  30 tr-l1: 0.02170 tr-p1: 19.70741 tr-l2: 0.08660 tr-p2: 65.19286
--------------------------------------------

  30 tr-l1: 0.03735 tr-p1: 0.10786 tr-l2: 0.08374 tr-p2: 3.05680
--------------------------------------------------------------------------------
epoch: 485
ood  loss 11.49183 acc: 0.231
e11 loss 1.89451 acc: 0.864
e22 loss 3.13081 acc: 0.785
--------------------------------------------------------------------------------
   0 tr-l1: 0.08186 tr-p1: 98.17036 tr-l2: 0.07636 tr-p2: 6.26980
  10 tr-l1: 0.05061 tr-p1: 6.97487 tr-l2: 0.05739 tr-p2: 7.13254
  20 tr-l1: 0.03031 tr-p1: 9.69119 tr-l2: 0.07051 tr-p2: 9.16455
  30 tr-l1: 0.02855 tr-p1: 7.25395 tr-l2: 0.10425 tr-p2: 100.36993
--------------------------------------------------------------------------------
epoch: 486
ood  loss 11.58109 acc: 0.224
e11 loss 1.91674 acc: 0.867
e22 loss 3.15131 acc: 0.785
--------------------------------------------------------------------------------
   0 tr-l1: 0.05552 tr-p1: 11.53977 tr-l2: 0.04559 tr-p2: 17.39836
  10 tr-l1: 0.04535 tr-p1: 1.91317 tr-l2: 0.06526 tr-p2: 0.00052
  20 tr-l1: 0.01863 tr